# Day6_3: 통합 실습 - 다중 소스 데이터 분석 (Multi-Source Data Integration)

## 📚 학습 목표

**Part 1: 기초**
1. CSV 파일과 SQLite 데이터베이스에서 데이터 로드하기
2. pandas merge를 사용한 데이터 통합 (inner/left/right/outer join)
3. groupby를 활용한 다차원 집계 분석
4. 집계 함수 조합 (sum, mean, count, max, min)
5. 결측치 처리 및 데이터 품질 검증

**Part 2: 심화**
1. Plotly Graph Objects로 복합 차트 생성
2. 서브플롯을 활용한 통합 대시보드 구성
3. 인터랙티브 기능 추가 (hover, 드릴다운)
4. HTML 리포트 자동 생성 및 배포

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 다중 소스 통합 | 서로 다른 시스템의 데이터를 결합 | CSV 주문 데이터 + DB 고객 정보 결합 |
| merge 조인 | 관계형 데이터 연결 | 주문 테이블 ↔ 고객 테이블 매칭 |
| groupby 집계 | 카테고리별 통계 분석 | 지역별/제품별 매출 집계 |
| Plotly 대시보드 | 경영진 보고용 인터랙티브 리포트 | 실시간 매출 현황판 |
| HTML 리포트 | 자동화된 리포팅 시스템 | 일일 매출 분석 리포트 발송 |

**분석가 관점**: 실무에서는 데이터가 하나의 파일에 모두 있는 경우가 드물며, CSV, 데이터베이스, API 등 여러 소스에서 데이터를 가져와 통합 분석하는 능력이 필수입니다.

---

## Part 1: 기초

### 1.1 실습 환경 준비

실무 시나리오를 위한 샘플 데이터를 생성합니다.

In [ ]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime, timedelta

print("✅ 라이브러리 임포트 완료")
print(f"pandas 버전: {pd.__version__}")

### 1.2 CSV 데이터 생성 (주문 데이터)

전자상거래 주문 데이터를 CSV 파일로 생성합니다.

In [ ]:
# 주문 데이터 생성 (orders.csv)
np.random.seed(42)

# 날짜 범위 생성 (최근 3개월)
start_date = datetime(2025, 10, 1)
dates = [start_date + timedelta(days=x) for x in range(90)]

# 200건의 주문 생성
orders_data = {
    'order_id': [f'ORD{i:04d}' for i in range(1, 201)],
    'customer_id': np.random.choice(range(1001, 1051), 200),  # 50명의 고객
    'product_id': np.random.choice(range(1, 21), 200),  # 20개의 제품
    'order_date': np.random.choice(dates, 200),
    'quantity': np.random.randint(1, 6, 200),
    'unit_price': np.random.choice([29000, 39000, 49000, 59000, 79000, 99000], 200)
}

orders_df = pd.DataFrame(orders_data)
orders_df['total_amount'] = orders_df['quantity'] * orders_df['unit_price']

# CSV 파일로 저장
orders_df.to_csv('orders.csv', index=False, encoding='utf-8-sig')

print("📁 orders.csv 파일 생성 완료")
print(f"\n주문 데이터 샘플 ({len(orders_df)}건):")
print(orders_df.head())
print(f"\n데이터 타입:\n{orders_df.dtypes}")

### 💡 실무 예시: CSV 데이터 로드

실무에서는 주문 데이터가 POS 시스템, 쇼핑몰 플랫폼 등에서 CSV로 추출되는 경우가 많습니다.

---

### 1.3 SQLite 데이터베이스 생성 (고객 및 제품 정보)

고객과 제품 마스터 데이터를 SQLite DB에 저장합니다.

In [ ]:
# SQLite 데이터베이스 연결
conn = sqlite3.connect('ecommerce.db')

# 고객 테이블 생성
customers_data = {
    'customer_id': range(1001, 1051),
    'customer_name': [f'고객{i}' for i in range(1, 51)],
    'region': np.random.choice(['서울', '경기', '부산', '대구', '인천'], 50),
    'membership_level': np.random.choice(['Bronze', 'Silver', 'Gold', 'Platinum'], 50),
    'join_date': pd.date_range(start='2024-01-01', periods=50, freq='W')
}

customers_df = pd.DataFrame(customers_data)
customers_df.to_sql('customers', conn, if_exists='replace', index=False)

print("✅ customers 테이블 생성 완료")
print(customers_df.head())

# 제품 테이블 생성
products_data = {
    'product_id': range(1, 21),
    'product_name': [f'상품{chr(65+i)}' for i in range(20)],
    'category': np.random.choice(['전자제품', '의류', '식품', '도서'], 20),
    'brand': np.random.choice(['브랜드A', '브랜드B', '브랜드C', '브랜드D'], 20),
    'stock_quantity': np.random.randint(10, 500, 20)
}

products_df = pd.DataFrame(products_data)
products_df.to_sql('products', conn, if_exists='replace', index=False)

print("\n✅ products 테이블 생성 완료")
print(products_df.head())

conn.close()
print("\n📊 ecommerce.db 데이터베이스 생성 완료")

### 💡 실무 예시: 데이터베이스 마스터 데이터

고객 정보와 제품 정보는 변경이 적고 여러 시스템에서 참조하므로 데이터베이스에 마스터 데이터로 관리하는 것이 일반적입니다.

---

### 1.4 데이터 로드 및 확인

이제 CSV와 DB에서 데이터를 로드하여 확인합니다.

In [ ]:
# CSV 파일 로드
orders = pd.read_csv('orders.csv', parse_dates=['order_date'])
print("📁 CSV 데이터 로드 완료")
print(f"주문 데이터: {orders.shape}")
print(orders.head(3))

# SQLite 데이터베이스 로드
conn = sqlite3.connect('ecommerce.db')
customers = pd.read_sql('SELECT * FROM customers', conn)
products = pd.read_sql('SELECT * FROM products', conn)
conn.close()

print("\n📊 DB 데이터 로드 완료")
print(f"고객 데이터: {customers.shape}")
print(f"제품 데이터: {products.shape}")

---

### 1.5 데이터 통합 (merge)

주문 데이터에 고객 정보와 제품 정보를 결합합니다.

In [ ]:
# 주문 + 고객 정보 결합 (left join)
orders_with_customer = orders.merge(
    customers,
    on='customer_id',
    how='left'
)

print("✅ 주문 + 고객 정보 결합 완료")
print(f"결합 후 데이터: {orders_with_customer.shape}")
print(orders_with_customer.head(3))

# 주문 + 고객 + 제품 정보 결합 (left join)
full_data = orders_with_customer.merge(
    products,
    on='product_id',
    how='left'
)

print("\n✅ 주문 + 고객 + 제품 정보 결합 완료")
print(f"최종 통합 데이터: {full_data.shape}")
print(full_data.head(3))
print(f"\n컬럼 목록:\n{full_data.columns.tolist()}")

### 💡 실무 예시: 데이터 조인 전략

- **left join**: 주문 데이터는 모두 유지하면서 고객/제품 정보 추가
- **inner join**: 매칭되는 데이터만 (고객 정보가 없는 주문 제외)
- **실무 선택**: 분석 목적에 따라 조인 방식 결정

---

### 1.6 데이터 품질 검증

통합된 데이터의 결측치와 중복을 확인합니다.

In [ ]:
# 결측치 확인
print("🔍 결측치 확인:")
missing_counts = full_data.isnull().sum()
print(missing_counts[missing_counts > 0])

if missing_counts.sum() == 0:
    print("✅ 결측치 없음")

# 중복 확인
print(f"\n🔍 중복 주문 확인: {full_data.duplicated(subset=['order_id']).sum()}건")

# 데이터 요약 통계
print("\n📊 수치형 데이터 요약:")
print(full_data[['quantity', 'unit_price', 'total_amount']].describe())

---

### 1.7 기본 groupby 집계

지역별, 카테고리별 매출을 집계합니다.

In [ ]:
# 지역별 매출 집계
region_sales = full_data.groupby('region')['total_amount'].agg([
    ('총매출', 'sum'),
    ('평균주문금액', 'mean'),
    ('주문건수', 'count')
]).round(0)

print("📊 지역별 매출 집계:")
print(region_sales.sort_values('총매출', ascending=False))

# 제품 카테고리별 매출 집계
category_sales = full_data.groupby('category')['total_amount'].agg([
    ('총매출', 'sum'),
    ('평균주문금액', 'mean'),
    ('주문건수', 'count')
]).round(0)

print("\n📊 카테고리별 매출 집계:")
print(category_sales.sort_values('총매출', ascending=False))

### 💡 실무 예시: 집계 함수 조합

- `sum`: 총 매출
- `mean`: 평균 주문 금액 (객단가)
- `count`: 주문 건수 (트래픽)
- 실무에서는 이 3가지를 조합하여 비즈니스 인사이트 도출

---

### 1.8 다차원 집계 (Multi-Level Groupby)

지역 + 카테고리별 교차 분석을 수행합니다.

In [ ]:
# 지역 + 카테고리별 매출 집계
multi_level = full_data.groupby(['region', 'category'])['total_amount'].sum().reset_index()
multi_level.columns = ['지역', '카테고리', '총매출']
multi_level = multi_level.sort_values('총매출', ascending=False)

print("📊 지역 + 카테고리별 매출 (Top 10):")
print(multi_level.head(10))

# 피벗 테이블로 변환
pivot_sales = multi_level.pivot(index='지역', columns='카테고리', values='총매출').fillna(0)

print("\n📊 피벗 테이블 (지역 × 카테고리):")
print(pivot_sales.astype(int))

---

## Part 2: 심화

### 2.1 Plotly Graph Objects 기본

막대 차트와 선 그래프를 Graph Objects로 생성합니다.

In [ ]:
# 지역별 매출 막대 차트
fig1 = go.Figure()

fig1.add_trace(go.Bar(
    x=region_sales.index,
    y=region_sales['총매출'],
    text=region_sales['총매출'].apply(lambda x: f'{x/10000:.0f}만원'),
    textposition='auto',
    marker_color='lightblue',
    name='총매출'
))

fig1.update_layout(
    title='지역별 총매출',
    xaxis_title='지역',
    yaxis_title='매출액 (원)',
    template='plotly_white',
    height=400
)

fig1.show()
print("✅ 지역별 매출 차트 생성 완료")

### 💡 실무 예시: Graph Objects의 장점

- **세밀한 커스터마이징**: 모든 요소 개별 제어 가능
- **복합 차트**: 여러 trace를 자유롭게 조합
- **프로덕션 대시보드**: 기업용 리포팅에 적합

---

### 2.2 시간별 트렌드 분석

일별 매출 추이를 선 그래프로 시각화합니다.

In [ ]:
# 일별 매출 집계
daily_sales = full_data.groupby('order_date')['total_amount'].sum().reset_index()
daily_sales.columns = ['날짜', '매출']

# 7일 이동평균 추가
daily_sales['이동평균'] = daily_sales['매출'].rolling(window=7, min_periods=1).mean()

# 선 그래프 생성
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=daily_sales['날짜'],
    y=daily_sales['매출'],
    mode='lines+markers',
    name='일별 매출',
    line=dict(color='lightblue', width=1),
    marker=dict(size=4)
))

fig2.add_trace(go.Scatter(
    x=daily_sales['날짜'],
    y=daily_sales['이동평균'],
    mode='lines',
    name='7일 이동평균',
    line=dict(color='red', width=2)
))

fig2.update_layout(
    title='일별 매출 트렌드',
    xaxis_title='날짜',
    yaxis_title='매출액 (원)',
    template='plotly_white',
    hovermode='x unified',
    height=400
)

fig2.show()
print("✅ 시계열 차트 생성 완료")

---

### 2.3 서브플롯 대시보드 (2×2 레이아웃)

4개의 차트를 하나의 대시보드로 통합합니다.

In [ ]:
# 서브플롯 생성 (2행 2열)
fig_dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=('지역별 총매출', '카테고리별 주문건수', 
                    '일별 매출 트렌드', '등급별 고객 분포'),
    specs=[[{'type': 'bar'}, {'type': 'pie'}],
           [{'type': 'scatter'}, {'type': 'bar'}]]
)

# 차트 1: 지역별 총매출 (막대)
fig_dashboard.add_trace(
    go.Bar(x=region_sales.index, y=region_sales['총매출'], 
           marker_color='lightblue', name='지역별 매출'),
    row=1, col=1
)

# 차트 2: 카테고리별 주문건수 (파이)
category_counts = full_data['category'].value_counts()
fig_dashboard.add_trace(
    go.Pie(labels=category_counts.index, values=category_counts.values, 
           name='카테고리'),
    row=1, col=2
)

# 차트 3: 일별 매출 트렌드 (선)
fig_dashboard.add_trace(
    go.Scatter(x=daily_sales['날짜'], y=daily_sales['이동평균'],
               mode='lines', name='7일 이동평균', line=dict(color='red', width=2)),
    row=2, col=1
)

# 차트 4: 등급별 고객 분포 (막대)
membership_counts = full_data['membership_level'].value_counts()
fig_dashboard.add_trace(
    go.Bar(x=membership_counts.index, y=membership_counts.values,
           marker_color='lightgreen', name='등급별 분포'),
    row=2, col=2
)

# 레이아웃 업데이트
fig_dashboard.update_layout(
    title_text='📊 전자상거래 통합 대시보드',
    showlegend=False,
    height=800,
    template='plotly_white'
)

fig_dashboard.show()
print("✅ 통합 대시보드 생성 완료")

### 💡 실무 예시: 대시보드 설계 원칙

1. **핵심 지표 우선**: 좌측 상단에 가장 중요한 차트 배치
2. **2×2 레이아웃**: 한눈에 4가지 인사이트 파악
3. **다양한 차트 타입**: 막대/파이/선 차트 조합으로 지루함 방지
4. **일관된 색상**: 브랜드 컬러 적용

---

### 2.4 인터랙티브 기능 추가

hover 정보를 커스터마이징하고 드릴다운을 구현합니다.

In [ ]:
# 지역별 상세 정보를 포함한 차트
region_detail = full_data.groupby('region').agg({
    'total_amount': 'sum',
    'order_id': 'count',
    'customer_id': 'nunique'
}).reset_index()

region_detail.columns = ['지역', '총매출', '주문건수', '고객수']

# 커스텀 hover 정보
fig_interactive = go.Figure()

fig_interactive.add_trace(go.Bar(
    x=region_detail['지역'],
    y=region_detail['총매출'],
    marker_color='lightcoral',
    customdata=region_detail[['주문건수', '고객수']],
    hovertemplate=
        '<b>%{x}</b><br>' +
        '총매출: %{y:,.0f}원<br>' +
        '주문건수: %{customdata[0]}건<br>' +
        '고객수: %{customdata[1]}명<br>' +
        '<extra></extra>'
))

fig_interactive.update_layout(
    title='지역별 매출 (상세 정보)',
    xaxis_title='지역',
    yaxis_title='총매출 (원)',
    template='plotly_white',
    height=400,
    hovermode='closest'
)

fig_interactive.show()
print("✅ 인터랙티브 차트 생성 완료")

---

### 2.5 HTML 리포트 자동 생성

대시보드를 HTML 파일로 저장하여 공유합니다.

In [ ]:
# HTML 리포트 생성
report_filename = f'sales_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.html'

# 대시보드를 HTML로 저장
fig_dashboard.write_html(report_filename)

print(f"✅ HTML 리포트 생성 완료: {report_filename}")
print(f"📁 파일 위치: {report_filename}")
print("\n💡 브라우저에서 열어 인터랙티브 대시보드를 확인하세요!")

### 💡 실무 예시: 자동화 리포팅

```python
# 실무에서는 이런 식으로 자동화
def generate_daily_report():
    # 1. 데이터 로드
    # 2. 집계 분석
    # 3. 차트 생성
    # 4. HTML 리포트 생성
    # 5. 이메일 자동 발송 (smtplib)
    pass

# 크론잡 또는 스케줄러로 매일 아침 8시 실행
```

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. CSV 파일 읽기 및 기본 탐색 ⭐

**문제**: `orders.csv` 파일을 읽고, 전체 주문 건수와 총 매출액을 출력하세요.

**기대 결과**: 
```
전체 주문 건수: 200건
총 매출액: XXXXXX원
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q2. SQLite 데이터 조회 ⭐

**문제**: `ecommerce.db`에서 `customers` 테이블을 읽고, 지역별 고객 수를 출력하세요.

**기대 결과**: 
```
지역
서울      XX명
경기      XX명
...
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q3. 데이터 merge 기본 ⭐⭐

**문제**: `orders`와 `products`를 `product_id`로 결합하여, 주문에 제품 이름(`product_name`)과 카테고리(`category`)를 추가하세요.

**기대 결과**: 결합된 DataFrame의 컬럼에 `product_name`, `category`가 포함됨

In [ ]:
# 여기에 코드를 작성하세요


---

### Q4. groupby 단일 집계 ⭐⭐

**문제**: `full_data`에서 제품 카테고리(`category`)별 평균 주문 금액을 계산하세요.

**기대 결과**: 
```
category
전자제품    XXXXX원
의류       XXXXX원
...
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q5. groupby 다중 집계 ⭐⭐⭐

**문제**: 등급(`membership_level`)별로 총 매출, 평균 주문 금액, 주문 건수를 한 번에 계산하세요.

**기대 결과**: 
```
                총매출   평균주문금액  주문건수
membership_level                          
Bronze          XXXXX      XXXXX       XX
...
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q6. 다차원 groupby ⭐⭐⭐

**문제**: 지역(`region`)과 등급(`membership_level`)별로 총 매출을 계산하고, 내림차순으로 정렬하세요.

**기대 결과**: 지역-등급 조합별 매출 순위

In [ ]:
# 여기에 코드를 작성하세요


---

### Q7. Plotly 막대 차트 생성 ⭐⭐⭐

**문제**: 브랜드(`brand`)별 총 매출을 막대 차트로 시각화하세요. (Graph Objects 사용)

**요구사항**:
- 막대 위에 매출액 텍스트 표시
- 제목: "브랜드별 총매출"
- 템플릿: `plotly_white`

In [ ]:
# 여기에 코드를 작성하세요


---

### Q8. 서브플롯 대시보드 ⭐⭐⭐⭐

**문제**: 1행 2열 레이아웃의 대시보드를 만드세요.
- 왼쪽: 지역별 주문 건수 (막대 차트)
- 오른쪽: 카테고리별 매출 비율 (파이 차트)

**기대 결과**: 2개의 차트가 나란히 배치된 대시보드

In [ ]:
# 여기에 코드를 작성하세요


---

### Q9. 커스텀 hover 정보 ⭐⭐⭐⭐

**문제**: 카테고리별 매출 막대 차트를 만들되, hover 시 다음 정보를 표시하세요:
- 카테고리명
- 총매출
- 주문건수
- 평균 주문금액

**힌트**: `customdata`와 `hovertemplate` 사용

In [ ]:
# 여기에 코드를 작성하세요


---

### Q10. 통합 분석 리포트 ⭐⭐⭐⭐⭐

**문제**: 다음 요구사항을 만족하는 종합 분석을 수행하고 대시보드를 생성하세요.

**요구사항**:
1. 월별 매출 추이 (선 그래프)
2. Top 5 제품의 매출 (막대 차트)
3. 지역별 매출 비율 (파이 차트)
4. 등급별 평균 주문 금액 (막대 차트)
5. 2×2 서브플롯으로 구성
6. HTML 파일로 저장 (`comprehensive_report.html`)

**기대 결과**: 4개 차트가 포함된 HTML 리포트 파일 생성

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/문법 | 실무 활용 |
|------|----------------|----------|
| CSV 로드 | `pd.read_csv(파일명, parse_dates=['날짜컬럼'])` | POS, 쇼핑몰 주문 데이터 |
| DB 로드 | `pd.read_sql('SELECT * FROM 테이블', conn)` | 고객/제품 마스터 데이터 |
| 데이터 통합 | `df1.merge(df2, on='키', how='left/inner')` | 주문-고객-제품 정보 결합 |
| 단일 집계 | `df.groupby('컬럼')['값'].sum()` | 카테고리별 매출 합계 |
| 다중 집계 | `df.groupby('컬럼').agg({'값': ['sum', 'mean']})` | 한 번에 여러 통계 계산 |
| 다차원 집계 | `df.groupby(['컬럼1', '컬럼2'])['값'].sum()` | 지역×카테고리 교차 분석 |
| 결측치 확인 | `df.isnull().sum()` | 데이터 품질 검증 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| Graph Objects 막대 | `go.Bar(x=[], y=[], text=[], textposition='auto')` | 세밀한 커스터마이징 필요시 |
| Graph Objects 선 | `go.Scatter(x=[], y=[], mode='lines+markers')` | 시계열 트렌드 시각화 |
| 서브플롯 | `make_subplots(rows=2, cols=2, specs=[...])` | 통합 대시보드 (한 화면에 여러 차트) |
| 커스텀 hover | `customdata=[], hovertemplate='...'` | 상세 정보 표시 (드릴다운) |
| HTML 저장 | `fig.write_html('리포트.html')` | 자동 리포팅, 이메일 발송 |
| 파이 차트 | `go.Pie(labels=[], values=[])` | 비율 분석 (카테고리 점유율) |
| 이동평균 | `df['값'].rolling(window=7).mean()` | 트렌드 평활화 |

### 💡 실무 팁

1. **데이터 통합 전략**: CSV는 트랜잭션, DB는 마스터 데이터로 역할 분담
2. **조인 선택**: `left join`이 가장 안전 (주문 데이터 손실 방지)
3. **집계 순서**: 단일 → 다중 → 다차원 순으로 점진적 분석
4. **대시보드 레이아웃**: 2×2가 가장 보편적 (한 화면에 4개 인사이트)
5. **HTML 리포트**: 스케줄러(cron, APScheduler)로 자동화 가능
6. **성능 최적화**: 대용량 데이터는 DB에서 집계 후 pandas로 시각화
7. **색상 일관성**: 브랜드 컬러 또는 `plotly_white` 템플릿 사용
8. **hover 정보**: 3-4개 핵심 지표만 표시 (과도한 정보는 역효과)

---

## 🎓 다음 학습

- **Day 7-0**: 정적 웹 스크레이핑 (BeautifulSoup)
- **Day 7-1**: 동적 웹 스크레이핑 (Selenium)
- **Week 2-3 통합 프로젝트**: 다중 소스 수집 + DB + 텍스트 분석

---